In [ ]:
## awareness-SIR modell und Funktionen

## Awareness File
This file includes the Awareness-SIR Model and the needed helper functions.<br> This file needs to import the <b>numpy</b> library to run the functions correctly. <br>
The awareness of an individual is calculated with the <b>awareness_SIR </b>- function.
Detailed aspects and code-design decisions are being explained in the function itself.
To get the final awareness values, functions to calculate the susceptibility and the awareness are also needed.

In [ ]:
# library imports
import numpy as np
from typing import Callable

In [ ]:
def susceptibility(rho: float, n: int) -> np.ndarray:  # factor which reduces susceptibility based on awareness
    """
    Determine how much the susceptibility will be reduced for each awareness compartment.

    The awareness level drops by a constant factor each time the information is transmitted one step further.
    The first compartment has a 100 % reduction in susceptibility.
    Only infected people can be in the very first compartment.
    The last compartment has zero awareness and therefore no susceptibility reduction.
    This way the sum of people in all compartments will be one.
    """
    # awareness is reduced by a constant factor for each distance it traveles
    # the last slot is for no awareness -> no susceptibility reduction
    if isinstance(rho, np.ndarray):
        sus = np.array([*[1-rho**i for i in range(n-1)], np.ones(shape=rho.shape)])
    else:
        sus = np.array([*[1-rho**i for i in range(n-1)], 1])
    return sus


In [ ]:
def g_awareness(rho: float, S: np.ndarray) -> np.ndarray:
    """
    Return the awareness level for an average S of a given SIR compartment.

    rho: information decay parameter (the quality of information is multiplied by this factor for each transmission)
    S: average of population, one SIR compartment, first axis must be information compartments
    """
    sus = susceptibility(rho, S.shape[0])
    if isinstance(rho, np.ndarray):
        a = np.sum(S*(1-sus), axis=0)
    else:
        a = np.sum(S*(1-sus[:, None]), axis=0)
    return a

In [ ]:
def awareness_SIR(t: float,y: np.ndarray, alpha: Callable[[float], float], beta: Callable[[float], float],\
    gamma: Callable[[float], float], omega: Callable[[float], float], lam: Callable[[float], float],\
    rho: Callable[[float], float], kappa: Callable[[float], float]) -> np.ndarray:
    """
    Return the derivative for a grid based awareness SIR model.

    Individuals are placed on a square lattice and only interact with their direct neighbors (up, down, left, right).
    Each individual has 3xA compartments where A denotes the number of awareness levels considered
    and the 3 comes from the S, I, R states of the SIR model.
    Thus, correlations between being infected and being informed are respected.
    However, correlations between one individual and their neighbors are not taken into account.


    t: time
    y: state of the compartments, dimensions: [i, j, SIR, awareness]
    information in indexing: https://numpy.org/doc/stable/user/basics.indexing.html

    The model parameters are given as functions of time and represent
    beta: virus transmission rate between two individuals
    gamma: recovery rate
    alpha: awareness transmission
    omega: awareness creation
    lam:   awareness fading over time
    rho:   susceptibility reduction (rh0=0.9 means 90% reduction for first level, 81% reduction for second level etc.)
    kappa: self isolation (kappa=0.8 means 80% reduction through isolation)

    returns: the derivative for the given state vector y at the given time t
    """

    # get parameters from possibly time dependant functions
    alpha = alpha(t)
    beta = beta(t)
    gamma = gamma(t)
    omega = omega(t)
    lam = lam(t)
    rho = rho(t)
    kappa = kappa(t)

    # initialize derivative as an empty array
    dy = np.zeros(shape=y.shape)  


    ### SIR model  ###

    ## change in infection and susceptibility

    # introduce shortcuts to access different parts of the grid for infecting neighbors
    upper = np.s_[:-1, :]  # whole grid except for the last row
    lower = np.s_[1:, :]  # everything except the first row
    left = np.s_[:, :-1]  # everything except the last column
    right = np.s_[:, 1:]  # everything except the first column

    origin = [upper, lower, left, right]  # to be iterated over
    destination = [lower, upper, right, left]  # "opposite" to origin, order must "match" origin

    ## infection transmission

    # iterate over the four directions the virus can spread
    for src, tgt in zip(origin, destination):

        # get reduced susceptibility based on awareness level
        sus = susceptibility(rho, y.shape[3])

        # get targets (susceptible ones)
        target = sus*y[tgt][..., 0, :]

        # get sources (infected ones, remove self isolations)
        source = (np.sum(y[src][..., 1, 1:], axis=-1) + (1-kappa)*y[src][..., 1, 0])[..., None]
        
        # move people from susceptible to infected
        dy[tgt][..., 1, :] += beta*target*source
        dy[tgt][..., 0, :] -= beta*target*source
        
    ## recovery ##

    # get infections
    I = y[..., 1, :]

    # move from infected to recovered
    dy[..., 1, :] -= gamma*I
    dy[..., 2, :] += gamma*I


    ### awareness ###

    ## information generation (acts on infected individuals)
    
    dy[..., 1, 0] += omega*np.sum(I, axis=-1)
    dy[..., 1, :] -= omega*I
    
    ## awareness transmission (lower index means higher awareness)

    # iterate over the four directions in which the awareness can spread
    for src, tgt in zip(origin, destination):
        
        # awareness increase

        # iterate over all awareness levels that people can move to
        for m in range(1, y.shape[3]-1):
            
            # get awareness transmission targets (all people with a awareness level index higher than m)
            target = np.sum(y[tgt][..., :, m+1:], axis=3)

            # get awareness transmission sources (all people with an awareness level index one lower than m)
            source = np.sum(y[src][..., :, m-1], axis=2)[..., None]

            # increase awareness for compartments with level m
            dy[tgt][..., :, m] += alpha*target*source
        
        # iterate over all awareness levels that people have moved away from
        for m in range(2, y.shape[3]):

            # get awareness transmission targets (people with awareness level m)
            target = y[tgt][..., :, m]

            # get awareness transmission sources (people with an awareness level index lower than m-1)
            source = np.sum(y[src][..., :, :m-1], axis=(2, 3))[..., None]

            # decrease awareness in the specified compartments with level m
            dy[tgt][..., :, m] -= alpha*target*source


    ## awareness fading
    dy[..., :, 1:] += lam*y[..., :, :-1]  # add awareness one level below
    dy[..., :, :-1] -= lam*y[..., :, :-1]  # and remove the moved ones

    return dy



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ac0f8ce2-3132-47be-a4d1-6216636e93ff' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>